In [1]:
import pandas as pd, numpy as np
import math

In [2]:
df = pd.read_csv('poke_genie_export.csv')

In [3]:
# You give a whole evolutionary line here.
mons = ['Spheal', 'Sealeo','Walrein']
mons = ['Bulbasaur', 'Ivysaur', 'Venusaur']
#mons = ['Venusaur', 'Venusaur', 'Venusaur']
mon = mons[-1]
base_ivs = {
    # base_attack, base_defense, base_stamina
    'Walrein': (182, 176, 242),
    'Trevenant': (201,154,198),
    'Venusaur': (198, 189, 190)
}[mon]
df = df[df.Name.isin(mons)]

In [4]:
df

,Index,Name,Form,Pokemon Number,Gender,CP,HP,Atk IV,Def IV,Sta IV,...,Sha/Pur (U),Rank % (L),Rank # (L),Stat Prod (L),Dust Cost (L),Candy Cost (L),Name (L),Form (L),Sha/Pur (L),Marked for PvP use
0,1,Venusaur,Normal,3,♂,2498,159,0.0,13.0,12.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Venusaur,Normal,3,♂,2489,153,14.0,14.0,12.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Venusaur,Normal,3,NaN,2058,139,8.0,13.0,8.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Venusaur,Normal,3,♂,1829,136,3.0,15.0,14.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Venusaur,Normal,3,♂,1546,121,15.0,14.0,14.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,Venusaur,Normal,3,NaN,1499,122,6.0,11.0,12.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,Venusaur,Normal,3,NaN,1151,105,15.0,12.0,13.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,Venusaur,Normal,3,NaN,1132,104,11.0,14.0,12.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,Venusaur,Normal,3,♂,934,93,6.0,12.0,4.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,Ivysaur,NaN,2,♂,761,89,8.0,10.0,8.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.columns

Index(['Index', 'Name', 'Form', 'Pokemon Number', 'Gender', 'CP', 'HP',
       'Atk IV', 'Def IV', 'Sta IV', 'IV Avg', 'Level Min', 'Level Max',
       'Quick Move', 'Charge Move', 'Charge Move 2', 'Scan Date',
       'Original Scan Date', 'Catch Date', 'Weight', 'Height', 'Lucky',
       'Shadow/Purified', 'Favorite', 'Dust', 'Rank % (G)', 'Rank # (G)',
       'Stat Prod (G)', 'Dust Cost (G)', 'Candy Cost (G)', 'Name (G)',
       'Form (G)', 'Sha/Pur (G)', 'Rank % (U)', 'Rank # (U)', 'Stat Prod (U)',
       'Dust Cost (U)', 'Candy Cost (U)', 'Name (U)', 'Form (U)',
       'Sha/Pur (U)', 'Rank % (L)', 'Rank # (L)', 'Stat Prod (L)',
       'Dust Cost (L)', 'Candy Cost (L)', 'Name (L)', 'Form (L)',
       'Sha/Pur (L)', 'Marked for PvP use'],
      dtype='object')

In [6]:
# I definitely stole this table from somewhere, and don't know where.
# If I remember, I'll give proper attribution.
def get_cpm(level):
    d = {
        1:	0.094,
        1.5: 0.1351374318,
        2: 0.16639787,
        2.5: 0.192650919,
        3: 0.21573247,
        3.5: 0.2365726613,
        4: 0.25572005,
        4.5: 0.2735303812,
        5: 0.29024988,
        5.5: 0.3060573775,
        6: 0.3210876,
        6.5: 0.3354450362,
        7: 0.34921268,
        7.5: 0.3624577511,
        8: 0.3752356,
        8.5: 0.387592416,
        9: 0.39956728,
        9.5: 0.4111935514,
        10: 0.4225,
        10.5: 0.4329264091,
        11: 0.44310755,
        11.5: 0.4530599591,
        12: 0.4627984,
        12.5: 0.472336093,
        13: 0.48168495,
        13.5: 0.4908558003,
        14: 0.49985844,
        14.5: 0.508701765,
        15: 0.51739395,
        15.5: 0.5259425113,
        16: 0.5343543,
        16.5: 0.5426357375,
        17: 0.5507927,
        17.5: 0.5588305862,
        18: 0.5667545,
        18.5: 0.5745691333,
        19: 0.5822789,
        19.5: 0.5898879072,
        20: 0.5974,
        20.5: 0.6048236651,
        21: 0.6121573,
        21.5: 0.6194041216,
        22: 0.6265671,
        22.5: 0.6336491432,
        23: 0.64065295,
        23.5: 0.6475809666,
        24: 0.65443563,
        24.5: 0.6612192524,
        25: 0.667934,
        25.5: 0.6745818959,
        26: 0.6811649,
        26.5: 0.6876849038,
        27: 0.69414365,
        27.5: 0.70054287,
        28: 0.7068842,
        28.5: 0.7131691091,
        29: 0.7193991,
        29.5: 0.7255756136,
        30: 0.7317,
        30.5: 0.7347410093,
        31: 0.7377695,
        31.5: 0.7407855938,
        32: 0.74378943,
        32.5: 0.7467812109,
        33: 0.74976104,
        33.5: 0.7527290867,
        34: 0.7556855,
        34.5: 0.7586303683,
        35: 0.76156384,
        35.5: 0.7644860647,
        36: 0.76739717,
        36.5: 0.7702972656,
        37: 0.7731865,
        37.5: 0.7760649616,
        38: 0.77893275,
        38.5: 0.7817900548,
        39: 0.784637,
        39.5: 0.7874736075,
        40: 0.7903,
        40.5: 0.792803968,
        41: 0.79530001,
        }
    return d[level]
get_cpm(2)

0.16639787

In [7]:
def ivs_to_stats(my_a, my_d, my_s, my_level, max_level=40,max_cp=1500.99, 
                 mon=mon, base_ivs=base_ivs):
    base_attack, base_defense, base_stamina = base_ivs
    attack = base_attack + my_a
    defense = my_d + base_defense
    stamina = my_s + base_stamina
    level = my_level#10
    cp = 10
    level_attack, level_defense, level_stamina, stat_prod = 0,0,0,0
    stats = (level, cp, level_attack, level_defense, level_stamina, stat_prod)
    while level <= max_level:
        cpm = get_cpm(level)
        cp = (attack * defense**0.5 * stamina**0.5 * cpm**2) / 10
        level_attack = attack * cpm
        level_defense = defense * cpm
        level_stamina = stamina * cpm
        if cp <= max_cp:
            stat_prod = math.floor(level_attack*level_defense*math.floor(level_stamina))
            stats = level, cp, level_attack, level_defense, level_stamina, stat_prod
        level = level + 0.5
    level, cp, level_attack, level_defense, level_stamina, stat_prod  = stats
    #print(f'{level_attack}, {level_defense}, {level_stamina}, {level_attack*level_defense*level_stamina}')
    return stats
    #print(f'level {level} cp {cp:.0f} attack {level_attack:.1f} defense {level_defense:.1f} stamina {level_stamina:.0f}')

In [8]:
ivs_to_stats(4,14,2,10)
ivs_to_stats(4,9,8,10)
ivs_to_stats(2,9,2,10)
ivs_to_stats(4,10,7,10)

(21.0,
 1498.7772296775097,
 123.6557746,
 121.81930270000001,
 120.59498810000001,
 1807639)

# Great League
## Note: max level is set to 40 in the line below. If you want to consider best buddies that aren't XL, set it to 41. If you want XL, set it to 50 or 51

In [9]:
d = {'CP':[],'max CP':[],'level':[],'attack':[],'defense':[],'stamina':[],'a':[],'d':[],'s':[],'statprod':[]}
for row in df.iterrows():
    i,s = row
    orig_cp, my_a, my_d, my_s, my_level = s.CP, s['Atk IV'], s['Def IV'], s['Sta IV'], s['Level Min']
    level, cp, attack, defense, stamina, stat_prod = ivs_to_stats(my_a, my_d, my_s,my_level = my_level, max_level=40)
    #print('orig_cp',orig_cp,'my_a',my_a)
    d['CP'].append(orig_cp)
    d['max CP'].append(int(np.floor(cp)))
    d['level'].append(level)
    d['attack'].append(attack)
    d['defense'].append(defense)
    d['stamina'].append(int(np.floor(stamina)))
    d['a'].append(my_a)
    d['d'].append(my_d)
    d['s'].append(my_s)
    d['statprod'].append(stat_prod)
mine = pd.DataFrame.from_dict(d)

In [10]:
# Walrein stats that meet the attack breakpoint
attack,defense,hp = 113.77, 113, 148
# Walrein stats that meet the GOD TIER
attack, defense, hp = 114.46,114.75, 148
# Walrein stats that hit all of the breakpoints
attack,defense,hp = 113.77, 113, 148
# Walrein great league high attack
attack, defense, hp = 114.46,113, 148
# Walrein great league waterfall
attack, defense, hp = 114.62, 113, 148


# For each of these, you do this
def get_mons(attack,defense,hp,mine,*,level_max=99):    
    these = mine[mine.attack >= attack]
    these = these[these.defense >= defense]
    these = these[these.stamina >= hp]
    these = these[these.level <= level_max]
    these = these.sort_values('statprod',ascending=False)
    return these

In [11]:
# Venusaur Frosslass Slayer OK def
# The 6/11/12 is a venu
attack, defense, hp = 122.5, 117.88, 122
get_mons(attack, defense, hp, mine).sort_values('defense',ascending=False)

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
5,1499,1499,20.5,123.384028,120.964733,122,6.0,11.0,12.0,1820864
18,267,1496,20.5,122.779204,120.359909,123,5.0,10.0,14.0,1817656
13,570,1488,20.5,123.384028,118.545438,122,6.0,7.0,13.0,1784446


In [12]:
# Venusaur Frosslass Slayer Big def
attack, defense, hp = 122.5, 121.13, 122
get_mons(attack, defense, hp, mine)

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod


In [13]:
# Venusaur Big Bulk
attack, defense, hp = 0, 121.13, 123
get_mons(attack, defense, hp, mine)

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod


In [14]:
# Worse version of that
attack, defense, hp = 0, 120.0, 123
get_mons(attack, defense, hp, mine)

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
18,267,1496,20.5,122.779204,120.359909,123,5.0,10.0,14.0,1817656


In [15]:
# Venusaur shadow big bulk, you manually check if it's a shadow
attack, defense, hp = 0, 120.46, 122
get_mons(attack, defense, hp, mine)

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
5,1499,1499,20.5,123.384028,120.964733,122,6.0,11.0,12.0,1820864


In [16]:
# Venusaur GL Mega Frosslass, needs to be level 15 or maybe 15.5. check.
attack, defense, hp = 0, 0, 105
get_mons(attack, defense, hp, mine, level_max=15.5)

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod


In [17]:
# Venusaur GL Mega Basti and Trev Atk
attack, defense, hp = 131.8, 0,0
get_mons(attack, defense, hp, mine,level_max=15)

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod


# UL requires recalculating
## Note: max level is set to 40 in the line below. If you want to consider best buddies that aren't XL, set it to 41. If you want XL, set it to 50 or 51

In [18]:
d = {'CP':[],'max CP':[],'level':[],'attack':[],'defense':[],'stamina':[],'a':[],'d':[],'s':[],'statprod':[]}
for row in df.iterrows():
    i,s = row
    orig_cp, my_a, my_d, my_s, my_level = s.CP, s['Atk IV'], s['Def IV'], s['Sta IV'], s['Level Min']
    level, cp, attack, defense, stamina, stat_prod = ivs_to_stats(my_a, my_d, my_s,max_level=40,my_level=my_level,max_cp=2500.99)
    #print('orig_cp',orig_cp,'my_a',my_a)
    d['CP'].append(orig_cp)
    d['max CP'].append(int(np.floor(cp)))
    d['level'].append(level)
    d['attack'].append(attack)
    d['defense'].append(defense)
    d['stamina'].append(int(np.floor(stamina)))
    d['a'].append(my_a)
    d['d'].append(my_d)
    d['s'].append(my_s)
    d['statprod'].append(stat_prod)
mine = pd.DataFrame.from_dict(d)

In [19]:
# Walrein UL supreme
attack, defense, hp = 147, 145.3, 197
# Walrein UL other ten
attack, defense, hp = 147.76, 143.89, 197
# Walrein UL hosing around
attack, defense, hp = 147.75, 145, 0


In [20]:
# Venusaur UL Gfisk Slayer
# The  6/12/4 is a shadow venu
attack, defense, hp = 159.2, 157.25, 152
get_mons(attack, defense, hp, mine)

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
2,2058,2499,38.0,160.460146,157.344415,154,8.0,13.0,8.0,3888116
8,934,2498,39.5,160.644616,158.282195,152,6.0,12.0,4.0,3864931


In [21]:
# Venusaur UL Huge Def
attack, defense, hp = 0,160.82, 156
get_mons(attack, defense, hp, mine)

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod


In [22]:
# Venusaur UL As good as Rank 1 bulk
# The 2498 is already a venu
attack, defense, hp = 0, 158.05, 156
get_mons(attack, defense, hp, mine)

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
0,2498,2498,40.0,156.479400,159.640600,159,0.0,13.0,12.0,3971893
3,1829,2487,38.0,156.565483,158.902281,158,3.0,15.0,14.0,3930820


In [23]:
# Venusaur UL Shadow General Blend (you check for shadow)
attack, defense, hp = 161.59, 152.28, 153
get_mons(attack, defense, hp, mine)

,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
6,1151,2495,35.0,162.213098,153.074332,154,15.0,12.0,13.0,3823921


# ML does not require recalculating because it's the raw numbers

In [24]:
# Walrein ML breakpoints
a,d,s = 15,14,13


In [25]:
# ML
#Want 15/14/13
print('Here are the ones that hit all of the BPs')
a,d,s = 15,14,13
these = mine[mine.a >= a]
these = these[these.d >= d]
these = these[these.s >= s]
these.sort_values('statprod',ascending=False)

Here are the ones that hit all of the BPs


,CP,max CP,level,attack,defense,stamina,a,d,s,statprod
4,1546,2494,34.5,161.588268,154.001965,154,15.0,14.0,14.0,3832276


### Some stuff left over from halloween that needs to be converted to the current format

In [26]:
# gfisk attack bp
these = pht[pht.attack >= 167.98]
# hp breakpoint for mirror
these = these[these.stamina >= 167]
these

NameError: name 'pht' is not defined

In [ ]:
# mew attack bp
these = pht[pht.attack >= 127.8]
these = these[these.defense >= 104.9] # best. next is 104.9.
# hp breakpoint for mirror
these = these[these.stamina >= 128]
these # 3/15/7 is RS, I think maybe 3/14/11 for Azu

In [ ]:
# mew attack bp
these = pht[pht.attack >= 127] 
these = these[these.defense >= 104.9] # best. next is 104.9.
# hp breakpoint for mirror
these = these[these.stamina >= 128]
these

In [ ]:
ivs_to_stats(my_a=3,my_d=14,my_s=11,max_cp=1501)